# RAS Profile Data Analysis

This notebook outlines a process for analyzing hydrological data stored in HDF5 format. 
It demonstrates how to:
- Generate a series of dates for a given period.
- Extract specific datasets from HDF5 files.
- Process and plot the extracted data for visualization.

The analysis focuses on two types of data: Water Surface and Flow rates at different reference lines within the study area.


## Imports and Setup

First, we import the necessary libraries for handling HDF5 files, numerical operations, and data visualization.


In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta


## Helper Functions for ARI Events

Several functions are defined to streamline the process of data extraction, processing, and visualization:

- `generate_dates`: Generates a list of datetime objects between two specified dates.
- `extract_names`: Extracts the names of reference lines from an HDF5 file.
- `extract_time_series`: Extracts time series data for Water Surface or Flow from an HDF5 file.
- `process_data`: Processes the extracted data for plotting.
- `plot_profile_time_series`: Creates and saves a plot of the time series data for a given reference line.


In [2]:
def generate_dates(start_date, end_date, num_steps):
    start = datetime.strptime(start_date, "%d %b %Y %H:%M")
    end = datetime.strptime(end_date, "%d %b %Y %H:%M")
    delta = (end - start) / (num_steps - 1)
    return [start + i * delta for i in range(num_steps)]


def extract_names(hdf_file_path, data_type):
    names_path = "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Name"
    with h5py.File(hdf_file_path, "r") as f:
        names_data = f[names_path][()]
        names = [name.split('|')[0] for name in names_data.astype(str)]
        return names


def extract_time_series(hdf_file_path, data_type):
    data_paths = {
        "Water Surface": "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Water Surface",
        "Flow": "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Flow"
    }
    with h5py.File(hdf_file_path, "r") as f:
        data_path = data_paths.get(data_type)
        if data_path in f:
            data = f[data_path][()]
            data = np.nan_to_num(data)
            return data
        else:
            raise KeyError(f"Data path '{data_path}' not found in the HDF file.")


def process_data(data, data_type):
    return data.T



def plot_profile_time_series(data, names, data_type, output_folder, dates):
    name_mapping = {
        "Kaouche_Coulee": "Kayouche Coulee",
        "CryingBrewery": "Crying Brewery",
        "GT": "Greinwich Terrace",
        "Near_LegionSt": "Near Legion St",
        "W_Sale_Rd": "West Sale Rd"
    }

    for idx, name in enumerate(names):
        if name in name_mapping:
            plt.figure(figsize=(10, 6), dpi=300)
            ax = plt.gca()

            data_to_plot = data[idx, :]

            plt.plot(dates, data_to_plot, label=f'{name_mapping[name]}', color='navy', linewidth=2)

            plot_title = f"{name_mapping[name]}"
            plt.title(plot_title, fontsize=20)
            plt.xlabel("Date", fontsize=16)
            plt.ylabel(f"{data_type.replace('_', ' ').title()} ({'ft³/s' if data_type == 'Flow' else 'ft'})", fontsize=16)

            plt.xticks(rotation=45)
            plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%d %b %Y %H:%M'))

            # Increase font size of tick labels for both axes
            ax.tick_params(axis='x', labelsize=14)
            ax.tick_params(axis='y', labelsize=14)

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('black')
            ax.spines['left'].set_linewidth(2)
            ax.spines['bottom'].set_color('black')
            ax.spines['bottom'].set_linewidth(2)

            ax.set_facecolor('white')
            ax.figure.set_facecolor('white')

            plt.tight_layout()

            file_name = f"{data_type}_{name.replace('_', '')}_time_series_profile.png"
            output_file_path = os.path.join(output_folder, file_name)
            plt.savefig(output_file_path, bbox_inches='tight', facecolor=ax.figure.get_facecolor())
            plt.close()



def main(hdf_file_base_path, output_base_folder, hdf_suffixes):
    start_date = "2 Jan 1970 00:00"
    end_date = "9 Jan 1970 12:00"


    for suffix in hdf_suffixes:
        hdf_file_path = f"{hdf_file_base_path}{suffix}.hdf"
        output_folder = f"{output_base_folder}_{suffix}"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for data_type in ["Water Surface", "Flow"]:
            names = extract_names(hdf_file_path, data_type)
            data = extract_time_series(hdf_file_path, data_type)
            data_processed = process_data(data, data_type)
            dates = generate_dates(start_date, end_date, data_processed.shape[1])
            plot_profile_time_series(data_processed, names, data_type, output_folder, dates)




## Main Procedure

The `main` function orchestrates the process of data extraction, processing, and visualization for each HDF5 file specified. 
It utilizes the helper functions to generate plots for each type of data (Water Surface and Flow) and saves them to the output directory.


## Plot all the ARI plans with single plots

In [3]:
if __name__ == "__main__":
    hdf_file_base_path = "S:\\For_Angshuman\\Greenbelt\\Task_ProfilePlots_2_27_2024\\Greenbelt_RAS.p"
    output_base_folder = "output_plots"
    hdf_suffixes = ["01", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "16", "18"]

    main(hdf_file_base_path, output_base_folder, hdf_suffixes)

## Saving the ARI dataframes

In [4]:
import pandas as pd
import os

def save_data_to_csv(data_processed, names, data_type, output_folder, dates, suffix, name_mapping):
    """
    Save the processed data into CSV files for each data type and suffix.

    Args:
    - data_processed (np.array): The processed data for all reference lines.
    - names (list): List of reference line names before mapping.
    - data_type (str): The type of data ('Flow' or 'Water Surface').
    - output_folder (str): The base folder where CSV files will be saved.
    - dates (list): List of datetime objects for each data point.
    - suffix (str): The HDF file suffix, indicating the specific dataset.
    - name_mapping (dict): A dictionary for mapping original names to desired names.
    """
    for idx, original_name in enumerate(names):
        if original_name in name_mapping:
            # Map the original name to the desired name
            friendly_name = name_mapping[original_name]
            # Create a DataFrame from the processed data
            df = pd.DataFrame(data_processed[idx, :], index=dates, columns=[friendly_name])
            df.index.name = 'Date'
            # Define the output CSV file path
            output_csv_path = os.path.join(output_folder, f"{suffix}_{data_type}_{friendly_name.replace(' ', '_')}.csv")
            # Save the DataFrame to CSV
            df.to_csv(output_csv_path)

def main(hdf_file_base_path, output_base_folder, hdf_suffixes):
    start_date = "2 Jan 1970 00:00"
    end_date = "9 Jan 1970 12:00"
    name_mapping = {
        "Kaouche_Coulee": "Kayouche Coulee",
        "CryingBrewery": "Crying Brewery",
        "GT": "Greinwich Terrace",
        "Near_LegionSt": "Near Legion St",
        "W_Sale_Rd": "West Sale Rd"
    }

    for suffix in hdf_suffixes:
        hdf_file_path = f"{hdf_file_base_path}{suffix}.hdf"
        output_folder_suffix = f"{output_base_folder}_{suffix}"
        
        if not os.path.exists(output_folder_suffix):
            os.makedirs(output_folder_suffix)

        for data_type in ["Water Surface", "Flow"]:
            names = extract_names(hdf_file_path, data_type)
            data = extract_time_series(hdf_file_path, data_type)
            data_processed = process_data(data, data_type)
            dates = generate_dates(start_date, end_date, data_processed.shape[1])
            
            # Save the data into CSV files instead of plotting
            save_data_to_csv(data_processed, names, data_type, output_folder_suffix, dates, suffix, name_mapping)

if __name__ == "__main__":
    hdf_file_base_path = "S:\\For_Angshuman\\Greenbelt\\Task_ProfilePlots_2_27_2024\\Greenbelt_RAS.p"
    output_base_folder = "dataframes_output"
    hdf_suffixes = ["01", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "16", "18"]

    if not os.path.exists(output_base_folder):
        os.makedirs(output_base_folder)

    main(hdf_file_base_path, output_base_folder, hdf_suffixes)


## Functions for couple plotting

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import glob

def format_title_legend(scenario):
    """
    Generate formatted title based on the scenario details.
    """
    # Split the scenario details
    parts = scenario.split('_')
    A, B, C, D = parts[0], parts[1], parts[2], parts[3]
    
    # Mapping for the year and tide parts
    year_tide_mapping = {
        "010yr": "10 year", "050yr": "50 year", "100yr": "100 year",
        "MeanTide50%": "Mean Tide 50%", "HighTide50%": "High Tide 50%"
    }
    
    # Format title
    year_formatted = year_tide_mapping.get(A, A)
    tide_formatted = year_tide_mapping.get(D, D)
    title = f"{year_formatted}, {B}, {tide_formatted}"
    
    return title

def determine_legend(suffix):
    """
    Determine legend label based on suffix.
    """
    with_project_suffixes = ['03', '04', '08', '09', '10', '11']
    return "With Project" if str(suffix).endswith(tuple(with_project_suffixes)) else "Existing Conditions"

def plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping):
    output_folder = "plots"
    os.makedirs(output_folder, exist_ok=True)
    
    for data_type in data_types:
        for scenario in scenarios:
            title = format_title_legend(scenario)
            for name_key, friendly_name in name_mapping.items():
                plt.figure(figsize=(10, 6), dpi=300)
                ax = plt.gca()

                legend_labels = set()
                for suffix in suffixes:
                    legend_label = determine_legend(suffix)
                    if legend_label in legend_labels:
                        continue
                    legend_labels.add(legend_label)

                    file_pattern = f"{base_folder}_{suffix}/{suffix}_{data_type}_{friendly_name.replace(' ', '_')}.csv"
                    files = glob.glob(file_pattern)
                    for file_path in files:
                        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
                        plt.plot(df.index, df.iloc[:, 0], label=legend_label, linewidth=2)

                plt.title(f"{friendly_name}: {title}", fontsize=20)
                plt.xlabel("Date", fontsize=16)
                plt.ylabel(f"{data_type.replace('_', ' ').title()} ({'ft³/s' if data_type == 'Flow' else 'ft'})", fontsize=16)
                plt.xticks(rotation=45)
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d %b %Y'))

                ax.tick_params(axis='x', labelsize=14)
                ax.tick_params(axis='y', labelsize=14)
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_color('black')
                ax.spines['left'].set_linewidth(2)
                ax.spines['bottom'].set_color('black')
                ax.spines['bottom'].set_linewidth(2)
                ax.set_facecolor('white')
                ax.figure.set_facecolor('white')

                plt.legend(fontsize=16)
                plt.tight_layout()

                plot_filename = os.path.join(output_folder, f"{data_type}_{friendly_name}_{title.replace(',', '').replace(' ', '_')}.png")
                plt.savefig(plot_filename)
                plt.close()



In [6]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['01', '03']  # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "010yr_24hr_Without_MeanTide50%",
    "010yr_24hr_WithProject_MeanTide50%"
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

In [7]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['05', '04']  # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "010yr_24hr_Without_HighTide50%",
    "010yr_24hr_WithProject_HighTide50%"
    
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

In [8]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['06', '09'] # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "050yr_24hr_Without_HighTide50%",
    "050yr_24hr_WithProject_HighTide50%"
    
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

In [9]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['07', '08'] # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "050yr_24hr_Without_MeanTide50%",
    "050yr_24hr_WithProject_MeanTide50%"
    
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

In [10]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['13', '10'] # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "100yr_24hr_Without_HighTide50%",
    "100yr_24hr_WithProject_HighTide50%"    
    
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

In [11]:
# Define your parameters for plotting
base_folder = 'dataframes_output'
suffixes = ['12', '11'] # Include all relevant suffixes
data_types = ['Flow', 'Water Surface']
scenarios = [
    "100yr_24hr_Without_MeanTide50%" ,
    "100yr_24hr_WithProject_MeanTide50%"
          
]

name_mapping = {
    "CryingBrewery": "Crying Brewery",
    "GT": "Greinwich Terrace",
    "Kaouche_Coulee": "Kayouche Coulee",
    "Near_LegionSt": "Near Legion St",
    "W_Sale_Rd": "West Sale Rd"
}
# Ensure the parameters for plotting are correctly defined as before

plot_data_pairs(base_folder, suffixes, data_types, scenarios, name_mapping)

## Helper Functions for May Event

In [12]:
def generate_dates(start_date, end_date, num_steps):
    start = datetime.strptime(start_date, "%d %b %Y %H:%M")
    end = datetime.strptime(end_date, "%d %b %Y %H:%M")
    delta = (end - start) / (num_steps - 1)
    return [start + i * delta for i in range(num_steps)]


def extract_names(hdf_file_path, data_type):
    names_path = "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Name"
    with h5py.File(hdf_file_path, "r") as f:
        names_data = f[names_path][()]
        names = [name.split('|')[0] for name in names_data.astype(str)]
        return names


def extract_time_series(hdf_file_path, data_type):
    data_paths = {
        "Water Surface": "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Water Surface",
        "Flow": "Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Lines/Flow"
    }
    with h5py.File(hdf_file_path, "r") as f:
        data_path = data_paths.get(data_type)
        if data_path in f:
            data = f[data_path][()]
            data = np.nan_to_num(data)
            return data
        else:
            raise KeyError(f"Data path '{data_path}' not found in the HDF file.")


def process_data(data, data_type):
    return data.T



def plot_profile_time_series(data, names, data_type, output_folder, dates):
    name_mapping = {
        "Kaouche_Coulee": "Kayouche Coulee",
        "CryingBrewery": "Crying Brewery",
        "GT": "Greinwich Terrace",
        "Near_LegionSt": "Near Legion St",
        "W_Sale_Rd": "West Sale Rd"
    }

    for idx, name in enumerate(names):
        if name in name_mapping:
            plt.figure(figsize=(10, 6), dpi=300)
            ax = plt.gca()

            data_to_plot = data[idx, :]

            plt.plot(dates, data_to_plot, label=f'{name_mapping[name]}', color='navy', linewidth=2)

            plot_title = f"{name_mapping[name]}"
            plt.title(plot_title, fontsize=20)
            plt.xlabel("Date", fontsize=16)
            plt.ylabel(f"{data_type.replace('_', ' ').title()} ({'ft³/s' if data_type == 'Flow' else 'ft'})", fontsize=16)

            plt.xticks(rotation=45)
            plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%d %b %Y %H:%M'))

            # Increase font size of tick labels for both axes
            ax.tick_params(axis='x', labelsize=14)
            ax.tick_params(axis='y', labelsize=14)

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('black')
            ax.spines['left'].set_linewidth(2)
            ax.spines['bottom'].set_color('black')
            ax.spines['bottom'].set_linewidth(2)

            ax.set_facecolor('white')
            ax.figure.set_facecolor('white')

            plt.tight_layout()

            file_name = f"{data_type}_{name.replace('_', '')}_time_series_profile.png"
            output_file_path = os.path.join(output_folder, file_name)
            plt.savefig(output_file_path, bbox_inches='tight', facecolor=ax.figure.get_facecolor())
            plt.close()



def main(hdf_file_base_path, output_base_folder, hdf_suffixes):
    start_date = "16 May 2021 00:00"
    end_date = "24 May 2021 12:00"


    for suffix in hdf_suffixes:
        hdf_file_path = f"{hdf_file_base_path}{suffix}.hdf"
        output_folder = f"{output_base_folder}_{suffix}"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for data_type in ["Water Surface", "Flow"]:
            names = extract_names(hdf_file_path, data_type)
            data = extract_time_series(hdf_file_path, data_type)
            data_processed = process_data(data, data_type)
            dates = generate_dates(start_date, end_date, data_processed.shape[1])
            plot_profile_time_series(data_processed, names, data_type, output_folder, dates)




## Plot all the May Events plans with single plots

In [13]:
if __name__ == "__main__":
    hdf_file_base_path = "S:\\For_Angshuman\\Greenbelt\\Task_ProfilePlots_2_27_2024\\Greenbelt_RAS.p"
    output_base_folder = "output_plots"
    hdf_suffixes = ["02", "15", "17"]

    main(hdf_file_base_path, output_base_folder, hdf_suffixes)

## Saving May Events Dataframes

In [14]:
import pandas as pd
import os

def save_data_to_csv(data_processed, names, data_type, output_folder, dates, suffix, name_mapping):
    """
    Save the processed data into CSV files for each data type and suffix.

    Args:
    - data_processed (np.array): The processed data for all reference lines.
    - names (list): List of reference line names before mapping.
    - data_type (str): The type of data ('Flow' or 'Water Surface').
    - output_folder (str): The base folder where CSV files will be saved.
    - dates (list): List of datetime objects for each data point.
    - suffix (str): The HDF file suffix, indicating the specific dataset.
    - name_mapping (dict): A dictionary for mapping original names to desired names.
    """
    for idx, original_name in enumerate(names):
        if original_name in name_mapping:
            # Map the original name to the desired name
            friendly_name = name_mapping[original_name]
            # Create a DataFrame from the processed data
            df = pd.DataFrame(data_processed[idx, :], index=dates, columns=[friendly_name])
            df.index.name = 'Date'
            # Define the output CSV file path
            output_csv_path = os.path.join(output_folder, f"{suffix}_{data_type}_{friendly_name.replace(' ', '_')}.csv")
            # Save the DataFrame to CSV
            df.to_csv(output_csv_path)

def main(hdf_file_base_path, output_base_folder, hdf_suffixes):
    start_date = "16 May 2021 00:00"
    end_date = "24 May 2021 12:00"
    name_mapping = {
        "Kaouche_Coulee": "Kayouche Coulee",
        "CryingBrewery": "Crying Brewery",
        "GT": "Greinwich Terrace",
        "Near_LegionSt": "Near Legion St",
        "W_Sale_Rd": "West Sale Rd"
    }

    for suffix in hdf_suffixes:
        hdf_file_path = f"{hdf_file_base_path}{suffix}.hdf"
        output_folder_suffix = f"{output_base_folder}_{suffix}"
        
        if not os.path.exists(output_folder_suffix):
            os.makedirs(output_folder_suffix)

        for data_type in ["Water Surface", "Flow"]:
            names = extract_names(hdf_file_path, data_type)
            data = extract_time_series(hdf_file_path, data_type)
            data_processed = process_data(data, data_type)
            dates = generate_dates(start_date, end_date, data_processed.shape[1])
            
            # Save the data into CSV files instead of plotting
            save_data_to_csv(data_processed, names, data_type, output_folder_suffix, dates, suffix, name_mapping)

if __name__ == "__main__":
    hdf_file_base_path = "S:\\For_Angshuman\\Greenbelt\\Task_ProfilePlots_2_27_2024\\Greenbelt_RAS.p"
    output_base_folder = "dataframes_output"
    hdf_suffixes = ["02", "15", "17"]

    if not os.path.exists(output_base_folder):
        os.makedirs(output_base_folder)

    main(hdf_file_base_path, output_base_folder, hdf_suffixes)


In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import glob

def plot_data_for_plans(base_folder, plans, data_types, name_mapping):
    output_folder = "plots_with_project_and_pond"
    os.makedirs(output_folder, exist_ok=True)
    
    for data_type in data_types:
        # Adjusting the data_type in the filename for "Water Surface" to include a space
        adjusted_data_type = data_type.replace('_', ' ') if data_type == "Water_Surface" else data_type
        
        for name_key, friendly_name in name_mapping.items():
            plt.figure(figsize=(10, 6), dpi=300)
            ax = plt.gca()

            for plan in plans:
                label = "No Pump" if plan == 15 else "Pump ON"
                # Make line style dotted for "No Pump"
                line_style = ":" if plan == 17 else "-"
                
                file_pattern = f"{base_folder}_{plan}/{plan}_{adjusted_data_type}_{friendly_name}.csv"
                files = glob.glob(file_pattern)
                if not files:
                    print(f"No files found for pattern: {file_pattern}")
                    continue
                for file_path in files:
                    df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
                    if not df.empty:
                        plt.plot(df.index, df.iloc[:, 0], label=label, linestyle=line_style, linewidth=2)
                    else:
                        print(f"Empty data frame for file: {file_path}")

            # Use friendly_name for the plot title
            plt.title(f"{friendly_name.replace('_', ' ')}: May 17, 2021 Event - With Project and Pond", fontsize=20)
            plt.xlabel("Date", fontsize=16)
            plt.ylabel(f"{data_type.replace('_', ' ').title()} ({'ft³/s' if data_type == 'Flow' else 'ft'})", fontsize=16)
            plt.xticks(rotation=45)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d %b %Y'))

            ax.tick_params(axis='x', labelsize=14)
            ax.tick_params(axis='y', labelsize=14)
            plt.legend(fontsize=16)
            plt.tight_layout()

            plot_filename = os.path.join(output_folder, f"{adjusted_data_type}_{friendly_name}_May_17_2021_Event.png")
            plt.savefig(plot_filename)
            plt.close()

# Parameters for plotting
base_folder = 'dataframes_output'
plans = [15, 17]  # Include Plan 15 and Plan 17
data_types = ['Flow', 'Water_Surface']  # Data types to plot, adjusted for correct spacing in filenames
name_mapping = {
    "CryingBrewery": "Crying_Brewery",
    "GT": "Greinwich_Terrace",
    "Kaouche_Coulee": "Kayouche_Coulee",
    "Near_LegionSt": "Near_Legion_St",
    "W_Sale_Rd": "West_Sale_Rd"
}

plot_data_for_plans(base_folder, plans, data_types, name_mapping)
